In [3]:
import tweepy

In [36]:
f = open('secret.txt', 'r')
mess = f.read().split('\n')
consumer_key = mess[0]
consumer_secret = mess[1]
access_token = mess[2]
access_token_secret = mess[3]
f.close()

In [37]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [47]:
tags = ['Trump', 'Donald']
lat = 40.730610
lon = -73.935242
rad = 10
since = '2015-12-21'
until = '2015-12-22'

def create_query_string(tags, since, until):
    st = " OR ".join(tags)
    st += (' since:%s' % since)
    st += (' until:%s' % until)
    return st

In [48]:
create_query_string(tags, since, until)

'Trump OR Donald since:2015-12-21 until:2015-12-22'

In [ ]:
create_geo(lat, lon, rad):
    return '%f,%f'